## TOPSIS  优劣解距离法
#### Technique for Order Preference by Similarity to an Ideal Solution

1. 将原始矩阵正向化（即将所有的指标类型转化为极大型指标，转换函数不唯一

    (1) 极小型指标  $x_{i}^{'} = max{\lbrace x_i \rbrace} - x_i$ 

    (2) 中间型指标  $M = max{\lbrace |x_i - x_{best}| \rbrace}$, $x_{i}^{'} = 1 - \frac{|x_i - x_{best}|}{M}$ 

    (3) 区间型指标  $M = max{\lbrace a - min{\lbrace x_i \rbrace},max{\lbrace x_i \rbrace} - b\rbrace}$,
    $x_{i}^{'} =\left\{ \begin{array}{l}
	1 - \frac{a-x}{M},x\leq a\\
	1,a\le x\le b\\
	1 - \frac{x-b}{M},x\geq b\\
    \end{array} \right.$

2. 正向化矩阵标准化

    $z_{ij} = x_{ij} / \sqrt{\sum_{i=1}^n {x_{ij}^2}}$

3. 计算得分并归一化

    定义最大值：$Z^{+} = (Z_1^{+}, Z_2^{+}, \dots, Z_m^{+}) = (max{\lbrace z_{11}, z_{21}, \dots, z_{n1} \rbrace}, max{\lbrace z_{12}, z_{22}, \dots, z_{n2} \rbrace}, \dots, max{\lbrace z_{1m}, z_{2m}, \dots, z_{nm} \rbrace})$

    定义最小值：$Z^{-} = (Z_1^{-}, Z_2^{-}, \dots, Z_m^{-}) = (min{\lbrace z_{11}, z_{21}, \dots, z_{n1} \rbrace}, min{\lbrace z_{12}, z_{22}, \dots, z_{n2} \rbrace}, \dots, min{\lbrace z_{1m}, z_{2m}, \dots, z_{nm} \rbrace})$

    定义第$i(i=1,2,\dots,n)$个评价对象与最大值的距离$D_{i}^{+}=\sqrt{\sum_{j=1}^{m}(Z_j^{+}-z_{ij})^2}$

    定义第$i(i=1,2,\dots,n)$个评价对象与最小值的距离$D_{i}^{-}=\sqrt{\sum_{j=1}^{m}(Z_j^{-}-z_{ij})^2}$

    那么我们可以得出第$i(i=1,2,\dots,n)$个评价对象未归一化的得分：$S_i=\frac{D_i^{-}}{D_i^{+}+D_i^{-}}$

    很明显$0 \leq S_i \leq 1$，且$S_i$越大$D_i^{+}$越大，即越接近最大值。

In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv("river.csv", header=None)
data

,0,1,2,3
0,4.69,6.59,51.0,11.94
1,2.03,7.86,19.0,6.46
2,9.11,6.31,46.0,8.91
3,8.61,7.05,46.0,26.43
4,7.13,6.50,50.0,23.57
5,2.39,6.77,38.0,24.62
6,7.69,6.79,38.0,6.01
7,9.30,6.81,27.0,31.57
8,5.45,7.62,5.0,18.46
9,6.19,7.27,17.0,7.51


In [5]:
class Topsis:
    def __init__(self, X, **typ):
        # 创建所使用数据的副本
        x_mat = X.copy()
        # 三种待转换的指标类型
        ctype = ['cmin', 'cmedian', 'crange']
        
        # typ 字典中提供了转换类型及相关列
        if typ:
            # 创建一个字典，仅包括极小型、中间型和区间型的指标类型及其相关列。
            type_dic = dict()
            for t in ctype:
                if t in typ.keys():
                    type_dic[t] = typ[t]
                    
            # 将 type_dic 中由需要转换的列组成的列表展平成一个单一的列表
            position = sum(type_dic.values(), [])

            for col_wait_for_convert in position:
                # 找到当前列对应的转换类型
                convert_type = [k for k, v in typ.items() if col_wait_for_convert in v][0]
                # 在给定的转换类型中找到当前列在待转换列列表中的索引
                current_index = typ[convert_type].index(col_wait_for_convert)
                
                # 中间型
                if convert_type == 'cmedian':
                    x_mat.iloc[:, col_wait_for_convert] = self.positivization(x_mat[col_wait_for_convert], convert_type,
                                                                              typ['best_median'][current_index])
                # 范围型
                elif convert_type == 'crange':
                    x_mat.iloc[:, col_wait_for_convert] = self.positivization(x_mat[col_wait_for_convert], convert_type,
                                                                              typ['best_range'][current_index])
                # 极小型
                else:
                    x_mat.iloc[:, col_wait_for_convert] = self.positivization(x_mat[col_wait_for_convert], convert_type)
        # typ 字典中未提供转换类型和列，则视为均已正向化
        else:
            print('无需正向化')
            
        self.x_mat = x_mat
    
    # 正向化函数
    def positivization(self, col, t, best=None):
        # 最小型
        if t == 'cmin':
            posit = col.max() - col
            return posit
        # 中间型
        elif t == 'cmedian':
            m = max(abs(col - best))
            posit = 1 - abs(col - best) / m
            return posit
        # 范围型
        elif t == 'crange':
            posit = col
            a,b = best
            m = max(np.append(a - min(col), max(col) - b))
            x_row = col.shape[0]
            for i in range(x_row):
                if col[i] < a:
                    posit[i] = 1 - (a - col[i]) / m
                elif col[i] > b:
                    posit[i] = 1 - (col[i] - b) / m
                else:
                    posit[i] = 1
            return posit
        
    # 标准化函数
    def standardize(self):
        # 每列元素平方和
        square_sum = np.square(self.x_mat).sum(axis=0)
        self.standard_mat = self.x_mat / np.sqrt(square_sum)
    
    # 计算距离及得分
    def score(self, weight=1):
        Z = self.standardize()
        col_max = Z.max()
        col_min = Z.min()
        D_max_sum = np.sqrt(np.sum(weight * (col_max-Z)**2, axis=1))
        D_min_sum = np.sqrt(np.sum(weight * (col_min-Z)**2, axis=1))
        S = D_min_sum / (D_max_sum + D_min_sum)
        stand_S = S / sum(S)
        
        self.stand_S = stand_S
        
        return self.standard_mat

In [9]:
tp = Topsis(X=data, cmin=[2], cmedian=[1], best_median=[7], crange=[3], best_range=[[10, 20]])
tp.score()
tp.stand_S

0     0.045058
1     0.047799
2     0.048497
3     0.048820
4     0.043113
5     0.044838
6     0.053947
7     0.050998
8     0.068074
9     0.068393
10    0.070162
11    0.059093
12    0.052652
13    0.019196
14    0.053307
15    0.043359
16    0.046574
17    0.043844
18    0.035810
19    0.056466
dtype: float64